Based on the baseline "Radek's LGBMRanker starter-pack”: https://www.kaggle.com/code/marcogorelli/radek-s-lgbmranker-starter-pack


# Helper functions

In [1]:
import numpy as np

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.

    This function computes the mean average prescision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists

    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

# https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/308635
def customer_hex_id_to_int(series):
    return series.str[-16:].apply(hex_id_to_int)

def hex_id_to_int(str):
    return int(str[-16:], 16)

def article_id_str_to_int(series):
    return series.astype('int32')

def article_id_int_to_str(series):
    return '0' + series.astype('str')

class Categorize(BaseEstimator, TransformerMixin):
    def __init__(self, min_examples=0):
        self.min_examples = min_examples
        self.categories = []

    def fit(self, X):
        for i in range(X.shape[1]):
            vc = X.iloc[:, i].value_counts()
            self.categories.append(vc[vc > self.min_examples].index.tolist())
        return self

    def transform(self, X):
        data = {X.columns[i]: pd.Categorical(X.iloc[:, i], categories=self.categories[i]).codes for i in range(X.shape[1])}
        return pd.DataFrame(data=data)


def calculate_apk(list_of_preds, list_of_gts):
    # for fast validation this can be changed to operate on dicts of {'cust_id_int': [art_id_int, ...]}
    # using 'data/val_week_purchases_by_cust.pkl'
    apks = []
    for preds, gt in zip(list_of_preds, list_of_gts):
        apks.append(apk(gt, preds, k=12))
    return np.mean(apks)

def eval_sub(sub_csv, skip_cust_with_no_purchases=True):
    sub=pd.read_csv(sub_csv)
    validation_set=pd.read_parquet('data/validation_ground_truth.parquet')

    apks = []

    no_purchases_pattern = []
    for pred, gt in zip(sub.prediction.str.split(), validation_set.prediction.str.split()):
        if skip_cust_with_no_purchases and (gt == no_purchases_pattern): continue
        apks.append(apk(gt, pred, k=12))
    return np.mean(apks)

In [3]:
import pandas as pd

# Read input files

In [4]:
%%time
pad = "/kaggle/input/makeparquet"
transactions = pd.read_parquet(pad+'/transactions_train.parquet')
customers = pd.read_parquet(pad+'/customers.parquet')
articles = pd.read_parquet(pad+'/articles.parquet')

CPU times: user 3.78 s, sys: 5.56 s, total: 9.34 s
Wall time: 6.95 s


In [5]:
test_week = transactions.week.max() + 1
transactions = transactions[transactions.week > transactions.week.max() - 10]

# Generating candidates

### Last purchase candidates

In [6]:
%%time

c2weeks = transactions.groupby('customer_id')['week'].unique()

CPU times: user 33.5 s, sys: 607 ms, total: 34.1 s
Wall time: 33.9 s


In [7]:
transactions.groupby('week')['t_dat'].agg(['min', 'max'])

,min,max
week,,
95,2020-07-15,2020-07-21
96,2020-07-22,2020-07-28
97,2020-07-29,2020-08-04
98,2020-08-05,2020-08-11
99,2020-08-12,2020-08-18
100,2020-08-19,2020-08-25
101,2020-08-26,2020-09-01
102,2020-09-02,2020-09-08
103,2020-09-09,2020-09-15


In [8]:
%%time

c2weeks2shifted_weeks = {}

for c_id, weeks in c2weeks.items():
    c2weeks2shifted_weeks[c_id] = {}
    for i in range(weeks.shape[0]-1):
        c2weeks2shifted_weeks[c_id][weeks[i]] = weeks[i+1]
    c2weeks2shifted_weeks[c_id][weeks[-1]] = test_week

CPU times: user 1.17 s, sys: 52.8 ms, total: 1.22 s
Wall time: 1.22 s


In [9]:
candidates_last_purchase = transactions.copy()

In [10]:
%%time

weeks = []
for i, (c_id, week) in enumerate(zip(transactions['customer_id'], transactions['week'])):
    weeks.append(c2weeks2shifted_weeks[c_id][week])

candidates_last_purchase.week=weeks

CPU times: user 32.9 s, sys: 218 ms, total: 33.2 s
Wall time: 33 s


### Bestsellers candidates

In [11]:
mean_price = transactions \
    .groupby(['week', 'article_id'])['price'].mean()

In [12]:
sales = transactions \
    .groupby('week')['article_id'].value_counts() \
    .groupby('week').rank(method='dense', ascending=False) \
    .groupby('week').head(12).rename('bestseller_rank').astype('int8')

In [13]:
bestsellers_previous_week = pd.merge(sales, mean_price, on=['week', 'article_id']).reset_index()
bestsellers_previous_week.week += 1

In [14]:
bestsellers_previous_week.pipe(lambda df: df[df['week']==96])

,week,article_id,bestseller_rank,price
0,96,760084003,1,0.025094
1,96,866731001,2,0.024919
2,96,600886001,3,0.022980
3,96,706016001,4,0.033197
4,96,372860002,5,0.013193
5,96,610776002,6,0.008318
6,96,877278002,7,0.025036
7,96,547780003,8,0.024814
8,96,817354001,9,0.021913
9,96,827968001,10,0.016436


In [15]:
unique_transactions = transactions \
    .groupby(['week', 'customer_id']) \
    .head(1) \
    .drop(columns=['article_id', 'price']) \
    .copy()

In [16]:
transactions.drop_duplicates(['week', 'customer_id'])

,t_dat,customer_id,article_id,price,sales_channel_id,season,year,month,week
29030503,2020-07-15,272412481300040,778064028,0.008458,1,2,-28,7,95
29064059,2020-07-15,1456826891333599,888294001,0.013542,1,2,-28,7,95
29067103,2020-07-15,2133687643102426,843642001,0.042356,2,2,-28,7,95
29027487,2020-07-15,6010692573790711,857812010,0.039661,1,2,-28,7,95
29046403,2020-07-15,6171059100114610,815447007,0.006763,2,2,-28,7,95
...,...,...,...,...,...,...,...,...,...
31760188,2020-09-22,18435221511488011015,573085055,0.033881,1,3,-28,9,104
31782234,2020-09-22,18436859303155335645,801447001,0.030492,1,3,-28,9,104
31787251,2020-09-22,18437941771381362708,907188001,0.050831,2,3,-28,9,104
31776022,2020-09-22,18438270306572912089,751471043,0.033881,1,3,-28,9,104


In [17]:
candidates_bestsellers = pd.merge(
    unique_transactions,
    bestsellers_previous_week,
    on='week',
)

In [18]:
test_set_transactions = unique_transactions.drop_duplicates('customer_id').reset_index(drop=True)
test_set_transactions.week = test_week

In [19]:
candidates_bestsellers_test_week = pd.merge(
    test_set_transactions,
    bestsellers_previous_week,
    on='week'
)

In [20]:
candidates_bestsellers = pd.concat([candidates_bestsellers, candidates_bestsellers_test_week])
candidates_bestsellers.drop(columns='bestseller_rank', inplace=True)

# Combining transactions and candidates / negative examples

In [21]:
transactions['purchased'] = 1

In [22]:
data = pd.concat([transactions, candidates_last_purchase, candidates_bestsellers])
data.purchased.fillna(0, inplace=True)

In [23]:
data.drop_duplicates(['customer_id', 'article_id', 'week'], inplace=True)

In [24]:
data.purchased.mean()

0.13607582749165664

### Add bestseller information

In [25]:
data = pd.merge(
    data,
    bestsellers_previous_week[['week', 'article_id', 'bestseller_rank']],
    on=['week', 'article_id'],
    how='left'
)

In [26]:
data = data[data.week != data.week.min()]
data.bestseller_rank.fillna(999, inplace=True)

In [27]:
data = pd.merge(data, articles, on='article_id', how='left')
data = pd.merge(data, customers, on='customer_id', how='left')

In [28]:
data.sort_values(['week', 'customer_id'], inplace=True)
data.reset_index(drop=True, inplace=True)

In [29]:
train = data[data.week != test_week]
test = data[data.week==test_week].drop_duplicates(['customer_id', 'article_id', 'sales_channel_id']).copy()

In [30]:
train_baskets = train.groupby(['week', 'customer_id'])['article_id'].count().values

In [31]:
columns_to_use = ['article_id', 'product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id',
'perceived_colour_master_id', 'department_no', 'index_code',
'index_group_no', 'section_no', 'garment_group_no', 'FN', 'Active',
'club_member_status', 'fashion_news_frequency', 'age', 'postal_code', 'bestseller_rank',
'buys_womens_clothing', 'buys_mens_clothes', 'buys_kid_baby_clothes', 'season',
'fabric_jeans','fabric_cotton', 'fabric_wool', 'fabric_polyester', 'fabric_silk', 'fabric_denim', 'fabric_linen', 'fabric_spandex', 'fabric_rayon', 'fabric_nylon', 'fabric_leather', 'fabric_suede']
#  'year', 'month',

In [32]:
%%time

train_X = train[columns_to_use]
train_y = train['purchased']

test_X = test[columns_to_use]

CPU times: user 643 ms, sys: 431 ms, total: 1.07 s
Wall time: 1.08 s


# Model training

In [33]:
from lightgbm.sklearn import LGBMRanker

In [34]:
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=1,
    importance_type='gain',
    verbose=10
)

In [35]:
%%time

ranker = ranker.fit(
    train_X,
    train_y,
    group=train_baskets,
)

[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.916047
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.509258
[LightGBM] [Debug] init for col-wise cost 1.528795 seconds, init for row-wise cost 7.896210 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 2.744739 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Sparse Multi-Val Bin
[LightGBM] [Info] Total Bins 1107
[LightGBM] [Info] Number of data points in the train set: 11381612, number of used features: 33
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
CPU times: user 51 s, sys: 7.84 s, total: 58.8 s
Wall time: 26 s


In [36]:
for i in ranker.feature_importances_.argsort()[::-1]:
    print(columns_to_use[i], ranker.feature_importances_[i]/ranker.feature_importances_.sum())

bestseller_rank 0.9959464973582318
season 0.0028884476828523694
buys_kid_baby_clothes 0.00048602297535493475
age 0.0002271913563462163
buys_mens_clothes 0.00017845578608163687
club_member_status 5.026178733690465e-05
postal_code 4.3260036177845566e-05
section_no 4.128856467573397e-05
product_type_no 3.940360791478877e-05
fabric_wool 2.833330143370025e-05
garment_group_no 2.145876785818854e-05
Active 1.6150356488640473e-05
perceived_colour_value_id 1.2832119006626338e-05
fashion_news_frequency 1.2347827731112598e-05
index_group_no 8.048472509456437e-06
department_no 0.0
index_code 0.0
perceived_colour_master_id 0.0
colour_group_code 0.0
graphical_appearance_no 0.0
FN 0.0
fabric_suede 0.0
fabric_leather 0.0
buys_womens_clothing 0.0
fabric_jeans 0.0
fabric_cotton 0.0
fabric_polyester 0.0
fabric_silk 0.0
fabric_denim 0.0
fabric_linen 0.0
fabric_spandex 0.0
fabric_rayon 0.0
fabric_nylon 0.0
article_id 0.0


# Calculate predictions

In [37]:
%time

test['preds'] = ranker.predict(test_X)

c_id2predicted_article_ids = test \
    .sort_values(['customer_id', 'preds'], ascending=False) \
    .groupby('customer_id')['article_id'].apply(list).to_dict()

bestsellers_last_week = \
    bestsellers_previous_week[bestsellers_previous_week.week == bestsellers_previous_week.week.max()]['article_id'].tolist()

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.58 µs


# Create submission with 3 veriosn: All predictions, only owned items, only new items

In [38]:
sub = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

In [39]:
subAltered = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

In [40]:
subOwned = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

In [41]:
# Create a dictionary of owned articles for each customer
owned_articles = transactions.groupby('customer_id')['article_id'].apply(set).to_dict()

# Define a function to replace owned articles with '000000000'
def replace_owned_articles(customer_id, predictions, owned_articles):
    new_predictions = []
    for article_id in predictions:
        if article_id not in owned_articles.get(customer_id, set()):
            new_predictions.append(article_id)
        else:
            new_predictions.append('000000000')
    return new_predictions

# Define a function to replace owned articles with '000000000'
def replace_unowned_articles(customer_id, predictions, owned_articles):
    new_predictions = []
    for article_id in predictions:
        if article_id in owned_articles.get(customer_id, set()):
            new_predictions.append(article_id)
        else:
            new_predictions.append('000000000')
    return new_predictions

In [42]:
%%time

preds = []
predsAlterd = []
predsOwned = []
for c_id in customer_hex_id_to_int(sub.customer_id):
    pred = c_id2predicted_article_ids.get(c_id, [])
        
    # Add bestsellers from the previous week
    pred = pred + bestsellers_last_week
        
    # Replace owned articles with '000000000'
    predAltered = replace_unowned_articles(c_id, pred, owned_articles)
    # Replace unowned articles with '000000000'
    predOwned = replace_owned_articles(c_id, pred, owned_articles)
    
    # Take the top 12 predictions (excluding any replaced '000000000' articles)
    preds.append(pred[:12])
    predsAlterd.append(predAltered[:12])
    predsOwned.append(predOwned[:12])


CPU times: user 35.2 s, sys: 557 ms, total: 35.8 s
Wall time: 35.8 s


In [43]:
preds = [' '.join(['0' + str(p) for p in ps]) for ps in preds]
sub.prediction = preds

In [44]:
sub_name = 'RepurchaseFilter_model_submissionUnAltered'
sub.to_csv(f'{sub_name}.csv.gz', index=False)

In [45]:
predsAlterd = [' '.join(['0' + str(p) for p in ps]) for ps in predsAlterd]
subAltered.prediction = predsAlterd

In [46]:
sub_nameAltered = 'RepurchaseFilter_model_submissionUnOwned'
subAltered.to_csv(f'{sub_nameAltered}.csv.gz', index=False)

In [47]:
predsOwned = [' '.join(['0' + str(p) for p in ps]) for ps in predsOwned]
subOwned.prediction = predsOwned

In [48]:
sub_nameOwned = 'RepurchaseFilter_model_submissionOwned'
subOwned.to_csv(f'{sub_nameOwned}.csv.gz', index=False)